### Import libraries 

In [43]:
# Import libraries
import re
import sys
import numpy as np
import pandas as pd
import numpy_financial as npf
import tkinter as tk
from tkinter.scrolledtext import ScrolledText

### Customize

In [44]:
# File names
my_output = "0701 cashflow_schedule.csv"

# User-defined Keywords
ASSET_VALUE_KEYWORDS = ["cost of asset", "cost of the asset", "cost of acquisition","initial cost", "purchase", "purchase price", "acquisition cost", "acquisition"]
ASSET_LIFE_KEYWORDS = ["life", "useful"]
ASSET_RESIDUAL_KEYWORDS = ["residual", "terminal"]

DEPRECIATION_METHOD_STRAIGHT_KEYWORDS = ["straight line", "slm"]
DEPRECIATION_METHOD_WRITTEN_DOWN_KEYWORDS = ["written down", "wdv"]

DEPRECIATION_RATE_KEYWORDS = ["depreciated", "depreciation rate", "rate of depreciation"]

ANNUAL_INFLOW_KEYWORDS = ["inflow", "income", "revenue", "revenues"]
ANNUAL_OUTFLOW_KEYWORDS = ["outflow", "cost", "expense", "expenses"]

TAX_RATE_KEYWORDS = ["tax"]
DISCOUNT_RATE_KEYWORDS = ["discount", "discounting", "cost of capital", "wacc"]

### Create extraction functions 

In [45]:
# EXTRACTION FUNCTIONS


def extract_numeric_value(text, keywords, symbol_prefix=None, symbol_suffix=None):
    """
    Searches for the first occurrence (after a given keyword) of a numeric value,
    optionally preceded by symbol_prefix and/or followed by symbol_suffix.
    Returns the float value if found; otherwise, None.
    """
    text_lower = text.lower()
    for kw in keywords:
        idx = text_lower.find(kw)
        if idx != -1:
            substring = text[idx:]
            if symbol_prefix and symbol_suffix:
                pattern = re.escape(symbol_prefix) + r'\s*([\d,\.]+)' + r'\s*' + re.escape(symbol_suffix)
            elif symbol_prefix:
                pattern = re.escape(symbol_prefix) + r'\s*([\d,\.]+)'
            elif symbol_suffix:
                pattern = r'([\d,\.]+)' + re.escape(symbol_suffix)
            else:
                pattern = r'([\d,\.]+)'
            match = re.search(pattern, substring)
            if match:
                try:
                    value_str = match.group(1).replace(',', '')
                    return float(value_str)
                except Exception:
                    continue
    return None

def extract_integer_value(text, keywords):
    """
    Searches for the first integer after one of the keywords.
    """
    text_lower = text.lower()
    for kw in keywords:
        idx = text_lower.find(kw)
        if idx != -1:
            substring = text_lower[idx:]
            match = re.search(r'(\d+)', substring)
            if match:
                try:
                    return int(match.group(1))
                except Exception:
                    continue
    return None

def extract_depreciation_method(text):
    """
    Checks for depreciation method keywords in the text.
    Returns "written_down" if any of its keywords are found,
    otherwise returns "straight_line" (even if no keyword is provided).
    """
    text_lower = text.lower()
    for kw in DEPRECIATION_METHOD_WRITTEN_DOWN_KEYWORDS:
        if kw in text_lower:
            return "written_down"
    for kw in DEPRECIATION_METHOD_STRAIGHT_KEYWORDS:
        if kw in text_lower:
            return "straight_line"
    return "straight_line"


### Create financial functions 

In [46]:

# ============================
# FINANCIAL FUNCTIONS
# ============================

def compute_payback(cashflows):
    """
    Compute the payback period by finding when cumulative cashflow becomes non-negative.
    If payback occurs between two years, linear interpolation is used.
    """
    cumulative = 0
    for i, cf in enumerate(cashflows):
        cumulative_prev = cumulative
        cumulative += cf
        if cumulative >= 0:
            if cf == 0:
                return i
            fraction = (0 - cumulative_prev) / cf
            return i - 1 + fraction
    return None

def compute_npv(cashflows, discount_rate):
    """
    Computes the NPV given a list of cashflows and an annual discount rate (in percent).
    """
    npv = 0
    dr = discount_rate / 100.0
    for t, cf in enumerate(cashflows):
        npv += cf / ((1 + dr) ** t)
    return npv

### Create functions for the main process

In [47]:
# ============================
# MAIN PROCESSING FUNCTION
# ============================

def process_input(text):
    # Step 1: Extract values
    asset_value    = extract_numeric_value(text, ASSET_VALUE_KEYWORDS, symbol_prefix='$')
    asset_life     = extract_integer_value(text, ASSET_LIFE_KEYWORDS)
    asset_residual = extract_numeric_value(text, ASSET_RESIDUAL_KEYWORDS, symbol_prefix='$')
    depreciation_method = extract_depreciation_method(text)
    depreciation_rate   = extract_numeric_value(text, DEPRECIATION_RATE_KEYWORDS, symbol_suffix='%')
    annual_inflow  = extract_numeric_value(text, ANNUAL_INFLOW_KEYWORDS, symbol_prefix='$')
    annual_outflow = extract_numeric_value(text, ANNUAL_OUTFLOW_KEYWORDS, symbol_prefix='$')
    tax_rate       = extract_numeric_value(text, TAX_RATE_KEYWORDS, symbol_suffix='%')
    discount_rate  = extract_numeric_value(text, DISCOUNT_RATE_KEYWORDS, symbol_suffix='%')

    if asset_residual is None:
        asset_residual = 0

    # For straight line method, set a default if depreciation_rate is missing.
    if depreciation_rate is None and depreciation_method == "straight_line" and asset_life:
        depreciation_rate = round(100 / asset_life)
    
    # Step 4: Display captured inputs
    print("Original Input:")
    print(text)
    print("-" * 40)
    print("Captured Inputs:")
    print(f"Asset Value: ${asset_value}")
    print(f"Asset Residual: ${asset_residual}")
    print(f"Asset Life: {asset_life} years")
    print(f"Depreciation Method: {depreciation_method}")
    print(f"Depreciation Rate: {depreciation_rate}%")
    print(f"Annual Inflow: ${annual_inflow}")
    print(f"Annual Outflow: ${annual_outflow}")
    print(f"Tax Rate: {tax_rate}%")
    print(f"Discount Rate: {discount_rate}%")
    print("-" * 40)

    # New condition: if using written down method and depreciation_rate is zero or not provided, exit.
    if depreciation_method == "written_down" and (depreciation_rate is None or depreciation_rate == 0):
        print("Cannot compute, key information missing: Depreciation rate for written down method is required. Please rephrase input")
        return

    if not asset_value or not annual_inflow or not annual_outflow or not discount_rate:
        print("Cannot compute, key information missing. Please rephrase input")
        return

    # Step 2: Compute depreciable value
    depreciable_value = asset_value - asset_residual

    # Build the cashflow schedule with columns in the required order.
    schedule = []
    # Year 0: initial investment row
    schedule.append({
        "Year": 0,
        "Inflow": 0,
        "Outflow": asset_value,
        "Depreciation": 0,
        "Tax": 0,
        "Profit_After_Tax": 0,
        "Annual_Cashflow": -asset_value
    })

    book_value = asset_value

    for year in range(1, asset_life + 1):
        if depreciation_method == "straight_line":
            annual_depreciation = depreciable_value / asset_life
        elif depreciation_method == "written_down":
            annual_depreciation = book_value * (depreciation_rate / 100.0)
            if (book_value - annual_depreciation) < asset_residual:
                annual_depreciation = book_value - asset_residual
        else:
            annual_depreciation = depreciable_value / asset_life

        book_value -= annual_depreciation

        # Compute annual income, tax, profit after tax, and cashflow.
        annual_income = annual_inflow - annual_outflow
        tax = annual_income * (tax_rate / 100.0) if tax_rate is not None else 0
        profit_after_tax = annual_income - tax
        cashflow = profit_after_tax + annual_depreciation

        if year == asset_life:
            cashflow += asset_residual

        schedule.append({
            "Year": year,
            "Inflow": annual_inflow,
            "Outflow": annual_outflow,
            "Depreciation": annual_depreciation,
            "Tax": tax,
            "Profit_After_Tax": profit_after_tax,
            "Annual_Cashflow": cashflow
        })

    # Ensure the DataFrame columns are in the desired order.
    df = pd.DataFrame(schedule)
    df = df[["Year", "Inflow", "Outflow", "Depreciation", "Tax", "Profit_After_Tax", "Annual_Cashflow"]]
    cashflows = df["Annual_Cashflow"].tolist()

    npv = compute_npv(cashflows, discount_rate)
    try:
        irr = npf.irr(cashflows)
        if np.isnan(irr):
            print("Warning: Could not compute IRR, saving it as Zero")
            irr = 0.0
        else:
            irr = irr * 100
    except Exception:
        print("Warning: Could not compute IRR, saving it as Zero")
        irr = 0.0

    payback = compute_payback(cashflows)
    payback_str = f"{payback:.2f} years" if payback is not None else "Not achieved"

    print("\nCashflow Schedule:")
    print(df.to_string(index=False))
    print("\nFinancial Metrics:")
    print(f"NPV: ${npv:,.2f}")
    print(f"IRR: {irr:.2f}%")
    print(f"Payback Period: {payback_str}")

    output_file = my_output
    df.to_csv(output_file, index=False)
    print(f"\nCashflow schedule saved to {output_file}")

    # Append additional financial metrics to the CSV file without altering the saved schedule.
    with open(output_file, "a") as f:
        f.write("\n")  # add an empty line for separation
        f.write("Metric,Value\n")
        f.write(f'NPV,"${npv:,.2f}"\n')
        f.write(f"IRR,{irr:.2f}%\n")
        f.write(f"Payback Period,{payback_str}\n")
        f.write(f"Discount Rate,{discount_rate}%\n")

### Create user input interface

In [48]:
# ============================
# GUI PROMPT FOR INPUT TEXT
# ============================

def prompt_for_input():
    root = tk.Tk()
    root.title("Enter Input Text")
    
    # Adjust the label to have word wrapping with a specific width
    label = tk.Label(
        root, 
        text="Please enter the input text below: Be sure to mention asset cost, depreciation rate, depreciation method, tax rate, expected life, residual value, annual revenue or cash inflow, average cost or cash outflow, etc. Remember all numeric values are to be preceded by $ sign and that this code works with predefined terms.",
        wraplength=600,  # Width in pixels for text wrapping
        justify=tk.LEFT  # Left-align the text
    )
    label.pack(pady=5, padx=15)
    
    # Your ScrolledText widget already has a predefined size (80x15)
    text_box = ScrolledText(root, width=80, height=15)
    text_box.pack(padx=10, pady=10)
    
    def on_process():
        input_text = text_box.get("1.0", tk.END).strip()
        if not input_text:
            tk.messagebox.showwarning("Input Error", "Input text cannot be empty!")
            return
        root.destroy()
        process_input(input_text)
    
    process_button = tk.Button(root, text="Process Input", command=on_process)
    process_button.pack(pady=10)
    
    root.mainloop()

if __name__ == "__main__":
    prompt_for_input()

Original Input:
Cost of asset is $500000 depreciation rate is 20% depreciation method is SLM tax rate is 50% residual value is $0 annual cash inflow is $120000 cash outflow is $ -5000 discounting rate is 10% useful life is -5 years
----------------------------------------
Captured Inputs:
Asset Value: $500000.0
Asset Residual: $0.0
Asset Life: 5 years
Depreciation Method: straight_line
Depreciation Rate: 20.0%
Annual Inflow: $120000.0
Annual Outflow: $500000.0
Tax Rate: 50.0%
Discount Rate: 10.0%
----------------------------------------

Cashflow Schedule:
 Year   Inflow  Outflow  Depreciation       Tax  Profit_After_Tax  Annual_Cashflow
    0      0.0 500000.0           0.0       0.0               0.0        -500000.0
    1 120000.0 500000.0      100000.0 -190000.0         -190000.0         -90000.0
    2 120000.0 500000.0      100000.0 -190000.0         -190000.0         -90000.0
    3 120000.0 500000.0      100000.0 -190000.0         -190000.0         -90000.0
    4 120000.0 500000.